In [17]:
# dependencies
import overpass
import pandas as pd
import requests
import json
from sqlalchemy import create_engine

# path for config file (project root folder)
import sys
sys.path.append('../../../../food-insecurity-machine-learning/')

from config import password, protocol, username, host, port, database_name


In [18]:
# Connection to AWS PostgresSQL Server
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [19]:
# get list of states and state fips codes
state_df = pd.read_sql_query('select * from state', engine)
state_df.head()

,StateFIPS,State
0,01,Alabama
1,02,Alaska
2,04,Arizona
3,05,Arkansas
4,06,California


In [20]:
# define overpass API query
api = overpass.API()
overpass_query = """
area[name="{state}"]->.a;
(
  node(area.a)["shop"="convenience"];
  node(area.a)["shop"="supermarket"];
  node(area.a)["shop"="grocery"];
  node(area.a)["shop"="health_food"];
  node(area.a)["shop"="organic"];
  node(area.a)["amenity"="marketplace"];
  way(area.a)["shop"="convenience"];
  way(area.a)["shop"="supermarket"];
  way(area.a)["shop"="grocery"];
  way(area.a)["shop"="health_food"];
  way(area.a)["shop"="organic"];
  way(area.a)["amenity"="marketplace"];
  relation(area.a)["shop"="convenience"];
  relation(area.a)["shop"="supermarket"];
  relation(area.a)["shop"="grocery"];
  relation(area.a)["shop"="health_food"];
  relation(area.a)["shop"="organic"];
  relation(area.a)["amenity"="marketplace"];
);
out;
"""

In [21]:
# loop over each state and retrieve food market data
for _, row in state_df.iterrows():
    # use state name in query
    query = overpass_query.format(state=row['State'])
    
    # make a get request to the overpass query
    response = api.get(query)
    
    # change to json
    data = response.json()
    
    # save data as a GeoJSON file
    with open(f'food_markets_{row["StateFIPS"]}.geojson', "w") as f:
        json.dump(data, f) 
    

UnknownOverpassError: Received corrupt data from Overpass (incomplete polygon).